# 2.0 CCLE Gene Expression Signature

In [29]:
from clustergrammer2 import net

In [30]:
import clustergrammer_groupby as cby
import warnings
warnings.filterwarnings('ignore')

In [31]:
import numpy as np
import pandas as pd
df = pd.read_csv('../data/CCLE/CCLE.txt.gz', compression='gzip', index_col=0)

In [34]:
df.shape

(18874, 1037)

In [32]:
from ast import literal_eval as make_tuple
cols = df.columns.tolist()
new_cols = [make_tuple(x) for x in cols]
df.columns = new_cols

In [33]:
all_genes = df.index.tolist()
len(all_genes)

18874

# CCLE Gene Expression Data, Z-score Genes

In [21]:
rows = df.index.tolist()
new_rows = [(x,) for x in rows]
df.index = new_rows
net.load_df(df)
net.filter_N_top(inst_rc='row', N_top=1000, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
df_tmp = net.export_df().round(2)
net.load_df(df_tmp)
net.cluster()
net.dendro_cats(axis='row', dendro_level=5)
df_dendro = net.export_df()
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 948, "rankvar": 689…

In [23]:
rows = df_dendro.index.tolist()
rows[0]

('KRT19', 'Group 5: cat-13')

In [35]:
gene_list = sorted([x[0] for x in rows if x[1] == 'Group 5: cat-5'])
len(gene_list)

121

### Tissue Type Signatures

In [7]:
gmts = {}
gmts['kea'] = net.load_gmt('../data/Enrichr_Libraries_of_Interest/KEA_2015.txt')

In [36]:
len(all_genes)

18874

In [37]:
len(set(all_genes).intersection(gene_list))

121

In [103]:
from scipy.stats import binom_test
def enrich_using_lib(lib_json, gene_list, background_list):
    len_gene_list = len(gene_list)
    
    list_terms = []
    list_pval = []
    # list of series that will be used to make dataframe
    list_term_ser = []
    for inst_term in lib_json:
        
        term_ser = pd.Series(data=np.zeros(len(gene_list)), index=gene_list)
        
        term_genes = lib_json[inst_term]
        
        p_expect = len(set(all_genes).intersection(term_genes))/len(all_genes)
        
        found_genes = list(set(gene_list).intersection(term_genes))
        actual_k = len(found_genes)
        
        # set found genes to one
        term_ser[found_genes] =  1
        
        if actual_k/len_gene_list > p_expect:
            inst_pval = binom_test(actual_k, len_gene_list, p_expect)
        else:
            inst_pval = 0.5
        
        if inst_pval < 0.05:
            
            list_terms.append(inst_term)
            list_pval.append(inst_pval)
            
        if actual_k > 0:
            term_name = (inst_term, 'Pval: ' + str(inst_pval))
            term_ser.name = term_name

            list_term_ser.append(term_ser)
            
    ser_pval = pd.Series(data=list_pval, index=list_terms).sort_values()
    df_term = pd.concat(list_term_ser, axis=1)
    
    # remove genes (rows) with no matches
    gene_sum = df_term.sum(axis=1)
    keep_genes = gene_sum[gene_sum > 0].index.tolist()
    df_term = df_term.loc[keep_genes]
    
    return ser_pval, df_term

In [109]:
ser_pval, df_term = enrich_using_lib(gmts['kea'], gene_list, all_genes)
ser_pval

LYN       9.367077e-07
SYK       4.175092e-06
PRKCD     4.553761e-04
HCK       2.438379e-03
IGF1R     1.720167e-02
TXK       1.905054e-02
PRKCQ     2.157760e-02
LCK       2.430763e-02
MAP4K1    2.532038e-02
FES       3.774109e-02
GRK6      3.774109e-02
KIT       3.774109e-02
FGR       4.389245e-02
MAPK9     4.796557e-02
dtype: float64

In [108]:
net.load_df(df_term)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "APOBEC3G", "ini": 34, "clust": 7, "rank": 11, "rankvar": 11, "…

In [88]:
len(gene_list)

121

In [7]:
df_sig, keep_genes_dict, df_gene_pval, fold_info = cby.generate_signatures(df,
                                                                     'tissue', num_top_dims=100)

net.load_df(df_sig)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "ABCA1", "ini": 703, "clust": 342, "rank": 341, "rankvar": 485,…

### Add tissue category to genes

In [8]:
gene_sig = df_sig.idxmax(axis=1)
gs_dict = {}
for inst_gene in gene_sig.index.tolist():
    gs_dict[inst_gene] = gene_sig[inst_gene][0]
    
rows = df.index.tolist()
new_rows = [(x, 'Cell Type: ' + gs_dict[x]) if x in gs_dict else (x, 'N.A.') for x in rows ]
df.index = new_rows
net.load_df(df)

In [9]:
ct_color = net.viz['cat_colors']['col']['cat-0']

In [10]:
def set_cat_colors(axis, cat_index, cat_title=False):
    for inst_ct in ct_color:
        if cat_title != False:
            cat_name = cat_title + ': ' + inst_ct
        else:
            cat_name = inst_ct
            
        inst_color = ct_color[inst_ct]
        net.set_cat_color(axis=axis, cat_index=cat_index, cat_name=cat_name, inst_color=inst_color)

In [11]:
set_cat_colors('row', 1)

### CCLE Data with Gene-Tissue Category

In [12]:
net.load_df(df)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 948, "rankvar": 689…